# 🔌 Conexión a PostgreSQL con Python
Este notebook implementa la Historia de Usuario HU1: Conexión y carga de datos desde PostgreSQL.

Usa un archivo `.env` para manejar credenciales de forma segura.

## 📦 Instalación de dependencias
Ejecuta esta celda para instalar las librerías necesarias.

In [ ]:
%pip install psycopg2-binary pandas sqlalchemy python-dotenv

## 🔐 Cargar variables del archivo `.env`
Debes crear un archivo `.env` en la misma carpeta del notebook con este contenido:

```
DB_HOST=localhost
DB_NAME=RIWI_VENTAS
DB_USER=postgres
DB_PASSWORD=123456
DB_PORT=5432
```


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

print("Variables cargadas correctamente.")

## 🧩 Clase de conexión a PostgreSQL

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

class ConexionPostgreSQL:
    """Clase para manejar conexión y extracción de datos desde PostgreSQL."""

    def __init__(self, host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        self.connection = None
        self.engine = None

    def conectar(self):
        try:
            self.connection = psycopg2.connect(
                host=self.host,
                database=self.database,
                user=self.user,
                password=self.password,
                port=self.port
            )

            conn_string = f"postgresql://{self.user}:{self.password}@{self.host}:{self.port}/{self.database}"
            self.engine = create_engine(conn_string)

            print("✅ Conexión exitosa a PostgreSQL")
            return True

        except Exception as e:
            print(f"❌ Error conectando: {e}")
            return False

    def listar_tablas(self):
        try:
            cursor = self.connection.cursor()
            cursor.execute("""
                SELECT table_name FROM information_schema.tables
                WHERE table_schema='public'
            """)
            tablas = cursor.fetchall()
            print("📋 Tablas en la base de datos:")
            for t in tablas:
                print(" -", t[0])
            return [t[0] for t in tablas]
        except Exception as e:
            print(f"❌ Error listando tablas: {e}")
            return []

    def extraer_tabla(self, nombre):
        try:
            df = pd.read_sql(f"SELECT * FROM {nombre}", self.engine)
            print(f"✅ '{nombre}' extraída: {df.shape[0]} filas")
            return df
        except Exception as e:
            print(f"❌ Error extrayendo {nombre}: {e}")
            return pd.DataFrame()

    def cerrar(self):
        if self.connection:
            self.connection.close()
            print("🔌 Conexión cerrada")

## ▶️ Ejecutar HU1

In [ ]:
def ejecutar_hu1():
    print("==============================")
    print(" HU1: Conexión y carga de datos")
    print("==============================")

    conexion = ConexionPostgreSQL()

    if conexion.conectar():
        conexion.listar_tablas()
        conexion.cerrar()

ejecutar_hu1()